<img src="https://i.imgur.com/6U6q5jQ.png"/>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/codes/blob/main/TheGeoDataFrame_intro.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
%reset
# starting fresh

# The Geo Dataframe

The geodataframe (GDF) is a dataframe (DF) where every row represents an spatial element (point, line, polygon).

Historically, the most common file type that stores spatial elements is the shapefile. Let's take a look at some of them:

1. Create a repository named: introgeodf.
2. Clone that repo to a local folder in your computer.
3. In that local folder in your computer, create a folder named **maps**.
4. Go to this [website](https://www.efrainmaps.es/english-version/free-downloads/world/).
5. Download three map files into the folder **maps** in your computer: *countries*, *cities*, and *rivers*.

You may see something like this:

<img src="https://github.com/CienciaDeDatosEspacial/code_and_data/blob/main/mapsFolderImage.png?raw=true">

You can decompress those files:

<img title="a title" alt="Alt text" src="https://github.com/CienciaDeDatosEspacial/code_and_data/blob/main/folderRar_1.png?raw=true">

Now, take a look a **World_Countries**:

<img src="https://github.com/CienciaDeDatosEspacial/code_and_data/blob/main/imageCountries_shp.png?raw=true">

There, you see that this **one map** requires **several files**. That is the nature of the shapefile.

Let's read the file with the help of **geopandas**:

In [ ]:
import os, geopandas as gpd

countries=gpd.read_file(os.path.join("maps","World_Countries","World_Countries.shp"))

Let's use some familiar DF functions:

In [ ]:
# what is it?
type(countries)

In [ ]:
# dimensions
countries.shape

In [ ]:
# names
countries.columns

In [ ]:
# some content
countries.head()

In [ ]:
# any missing values?
countries[countries.isna().any(axis=1)]

In [ ]:
# types
countries.info()

As you see, every pandas command is working, but now we have a new column type: **geometry**. Let's see this map of countries:

In [ ]:
countries.plot(facecolor="azure",#color of polygon fill
               edgecolor='black', #color of lines
               linewidth=0.1) #thickness of lines

Let's open the other maps:

In [ ]:
rivers=gpd.read_file(os.path.join("maps","World_Hydrography","World_Hydrography.shp"))
cities=gpd.read_file(os.path.join("maps","World_Cities","World_Cities.shp"))

This is the rivers map:

In [ ]:
rivers.plot(edgecolor='blue',
            linewidth=1,
            linestyle='dotted')

This is the cities map:

In [ ]:
cities.plot(marker='.', # marker type
            color='red',
            markersize=1,
            alpha=0.3) # transparency

You can plot all the layers, as long as they share the same projection.
Let's verify that all have the same projection (**CRS**):

In [ ]:
countries.crs==cities.crs==cities.crs

You can start by creating the layer on the back (the base), and add layers on top:

In [ ]:
base = countries.plot(facecolor="white",
                      edgecolor='black',
                      linewidth=0.1,
                      figsize=(12,12))

rivers.plot(edgecolor='blue', linewidth=0.4,
            ax=base)# on top of...
cities.plot(marker='.', color='red', markersize=1,alpha=0.7,
            ax=base) # on top of...


## Subsetting

You can subset your map by *filtering*:

In [ ]:
brazil=countries[countries.COUNTRY=='Brazil']

But you can also subset by *clipping*, as sometimes other data frames may not have the same fields for filtering:

In [ ]:
cities_clipped = gpd.clip(gdf=cities,
                          mask=brazil)
rivers_clipped = gpd.clip(gdf=rivers,
                               mask=brazil)

Then, you can plot the clipped version:

In [ ]:
base = brazil.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
cities_clipped.plot(marker='+', color='red', markersize=15,
                    ax=base)
rivers_clipped.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

You can also check what geometries you have in your GDF:

In [ ]:
brazil.geom_type

In [ ]:
cities_clipped.geom_type

In [ ]:
rivers_clipped.geom_type

Notice that the amount of elements (rows) is different, and that all those elements do not belong to the exact geometry type.

## Exporting your subset

You may need a couple of things here:

- Save the map as an image.
- Save the files (not the images).

Let's see:

In [ ]:
# save the map as image (create "images" folder first)
import matplotlib.pyplot as plt

base = brazil.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
cities=cities_clipped.plot(marker='+', color='red', markersize=15,ax=base)
river=rivers_clipped.plot(edgecolor='blue', linewidth=0.5,ax=base)
plt.savefig(os.path.join("images",'Brasil_3layers.jpg'))

We may export these layers as one different file (not shapefiles):

In [ ]:
brazil.to_file(os.path.join("maps","brazilMaps.gpkg"), layer='country', driver="GPKG")
cities_clipped.to_file(os.path.join("maps","brazilMaps.gpkg"), layer='cities', driver="GPKG")
rivers_clipped.to_file(os.path.join("maps","brazilMaps.gpkg"), layer='rivers', driver="GPKG")

The most important thing, now that you have saved these several maps into one file, is that once this file is stored in GitHub, you can call the map with the url GitHUb gives you:

In [ ]:
# commit and push **brazilMaps.gpkg** and get a link like:
brazilMaps='https://github.com/SocialAnalytics-StrategicIntelligence/codes/raw/main/maps/brazilMaps.gpkg'

### Reading in from the cloud:

Let's use the *brazilMaps* link, and verify we have all the layers:

In [ ]:
from  fiona import listlayers

listlayers(brazilMaps)

Now you are confident what to request:

In [ ]:
countryGit=gpd.read_file(brazilMaps,layer='country')
citiesGit=gpd.read_file(brazilMaps,layer='cities')
riversGit=gpd.read_file(brazilMaps,layer='rivers')

As you see, it works great:

In [ ]:
base = countryGit.plot(facecolor='gainsboro')
citiesGit.plot(ax=base, markersize=0.5, color='red')
riversGit.plot(ax=base, linewidth=0.5)

### Exercise 1

<div class="alert-success">
    
1. Create a repo: **simpleplot**
2. Clone that repo to your computer.
3. Get **three** _SHP_ files from the same **country** (do not use islands or small territories). You should have polygons (i.e. regions).
4. Save the three maps into one geopackage file.    
</div>



<a class="anchor" id="1"></a>

## Map Projection

The CRS is a very important property of the maps. They affect three aspects:

* shape
* area
* scale
* direction

Most maps come with a default CRS: 4326. Pay attention:

In [ ]:
# check units
countryGit.crs.axis_info

Polygons have a centroid. When we try getting a centroid from an **unprojected** polygon, you get: 

In [ ]:
# centroid
countryGit.centroid

### Reprojecting

A projected CRS will have units in meters or feet (or similar):

In [ ]:
# let's use CRS 3587

countryGit.to_crs(3587).crs.axis_info

The reprojected map will not complain if I request the centriod:

In [ ]:
countryGit.to_crs(3587).centroid

Let's plot both:

In [ ]:
# plot this projected version

base3587= countryGit.to_crs(3587).plot()
countryGit.to_crs(3587).centroid.plot(color='red',ax=base3587)

We use the crs **3587** as an emergency option to reproject a map. However, for a more accurate option it is better to look for the ones explicitly prepared for a particular locations of the world. You can request a crs per country [here](https://epsg.io/?q=brazil+kind%3APROJCRS):

In [ ]:
# recommended for Brazil (meters)
countryGit.to_crs(5641).crs.axis_info

In [ ]:
# this works
countryGit.to_crs(5641).centroid

In [ ]:
# replotting:

base5641=countryGit.to_crs(5641).plot()
countryGit.to_crs(5641).centroid.plot(color='red',ax=base5641)

Not using the right projection will give you a wrong numerical result when needing numerical accuracy; however, you might find situation where the visual output seems right (yet it is wrong):

In [ ]:
from matplotlib import pyplot

fig, (ax1, ax2) = pyplot.subplots(ncols=2, sharex=False, sharey=False, figsize=(12,12))

ax1.set_title('projected (5641)',fontsize=10)
countryGit.to_crs(5641).plot(ax=ax1)
countryGit.to_crs(5641).centroid.plot(color='red',ax=ax1)

# this gives a warning
ax2.set_title('unprojected',fontsize=10)
countryGit.plot(ax=ax2)
countryGit.centroid.plot(color='red',ax=ax2)

Let's keep the projected version for all our maps:

In [ ]:
brazil_5641=countryGit.to_crs(5641)
# just Brazil

cities_5641=citiesGit.to_crs(brazil_5641.crs)
cities_5641=cities_5641[cities_5641.COUNTRY=='Brazil']

rivers_5641=riversGit.to_crs(brazil_5641.crs)
rivers_5641=gpd.clip(gdf=rivers_5641,mask=brazil_5641)

We could save these maps of Brazil:

In [ ]:
# saving 
import os

brazil_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='country', driver="GPKG")
cities_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='cities', driver="GPKG")
rivers_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='rivers', driver="GPKG")

<a class="anchor" id="1_2"></a>

### Maps without CRS

Reprojecting seems a simple process, but you might find some interesting cases. Download the shapefile of Brazil - Subnational Administrative Boundaries from [here](https://data.humdata.org/dataset/cod-ab-bra) and save it in the maps folder (you need to unzip the file).

In [ ]:
brazil_states=gpd.read_file(os.path.join("maps","bra_adm_ibge_2020_shp","bra_admbnda_adm1_ibge_2020.shp"))
brazil_municipalities=gpd.read_file(os.path.join("maps","bra_adm_ibge_2020_shp","bra_admbnda_adm2_ibge_2020.shp"))

Notice this:

In [ ]:
brazil_states.crs, brazil_municipalities.crs

They do not have crs information, however:

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(ncols=2, sharex=False, sharey=False, figsize=(12,12))

brazil_states.plot(ax=ax1, facecolor='lightgrey', edgecolor='black')
brazil_municipalities.plot(ax=ax2, facecolor='lightgrey', edgecolor='black',linewidth=0.2)

Since we are using the crs 5641 for Brazil, we could try:

In [ ]:
brazil_states.to_crs(5641)

Python says _"Please set a crs on the object first"_. This would mean to know the actual projection, of the geometry:

In [ ]:
brazil_states.geometry

From the plots above, it looks like an unprojected map, then:

In [ ]:
brazil_states.crs = "EPSG:4326"
brazil_municipalities.crs = "EPSG:4326"

Now, we can reproject:

In [ ]:
brazil_states_5641=brazil_states.to_crs(5641)
brazil_municipalities_5641=brazil_municipalities.to_crs(5641)

We update the file:

In [ ]:
brazil_states_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='states', driver="GPKG")
brazil_municipalities_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='municipalities', driver="GPKG")

### Exercise 2

<div class="alert-success">

1. Get more file maps for your country (lower level administrative units).
2. Check all the CRSs of your GDFs.
3. If you find one CRS is missing, fill the CRS with the right projection.
4. If you have unprojected CRS, look for the right one and reset it.
</div>

<a class="anchor" id="1_1"></a>

### Formating Geoseries projections

You know **brazil_5641** is a multipolygon:

In [ ]:
brazil_5641

Sometime, you just need the border (lines):

In [ ]:
brazil_5641.boundary

In [ ]:
# This is just the borderline
brazil_5641.boundary.plot()

Always check the data type:

In [ ]:
# does 'boundary' return a GDF?
type(brazil_5641.boundary)

Some operations in geopandas require GDF or GS, but some may be exclusive for either. If you need a GDF instead of a GS:

In [ ]:
# converting into GDF
brazil_5641.boundary.to_frame()

Notice you get a very simple GDF, and you may want to add some info:

In [ ]:
# conversion
brazil_5641DF=brazil_5641.boundary.to_frame() 

# new column (optional)
brazil_5641DF['name']='Brazil' 

# renaming the geometry column
brazil_5641DF.rename(columns={0:'geometry'},inplace=True) 

#setting the geometry (the name is not enough)
brazil_5641DF = brazil_5641DF.set_geometry("geometry")

# verifying crs:
brazil_5641DF.crs

In [ ]:
# see it
brazil_5641DF

You can add this GDF as a layer:

In [ ]:
brazil_5641DF.to_file(os.path.join("maps","brazilMaps_5641.gpkg"), layer='border', driver="GPKG")

### Exercise 3

<div class="alert-success"> 
    
1. Recover the boundaries of your country GDF (polygon).
    
2. Turn the boundary into a GDF.
</div>

<a class="anchor" id="3"></a>

## Creating Spatial data

You will get Lines and Polygons as maps for sure, but that may not be the case with points. Let me download a **CSV** file with information on the airports in Brazil from this [website](https://data.humdata.org/dataset/ourairports-bra), I will save it in my **data** folder:

In [ ]:
import pandas as pd 
infoairports=pd.read_csv(os.path.join("data","br-airports.csv"))

# some rows

infoairports.iloc[[0,1,2,3,-4,-3,-2,-1],:] #head and tail

This needs some cleaning:

In [ ]:
# bye first row 
infoairports.drop(index=0,inplace=True)
infoairports.reset_index(drop=True, inplace=True)
infoairports.head()

In [ ]:
# keep the  columns needed

infoairports.columns

In [ ]:
keep=['name','type','latitude_deg', 'longitude_deg','elevation_ft','region_name','municipality']
infoairports=infoairports.loc[:,keep]

In [ ]:
infoairports.info()

Some formatting:

In [ ]:
numericCols=['latitude_deg', 'longitude_deg','elevation_ft']
infoairports[numericCols]=infoairports.loc[:,numericCols].apply(lambda x:pd.to_numeric(x))

# now 
infoairports.info()

In [ ]:
# let's plot

base = countryGit.plot(color='white', edgecolor='black') #unprojected

infoairports.plot.scatter(x = 'longitude_deg', y = 'latitude_deg',ax=base)

Would that be ok? It is supposed to be right. 
Let me turn those coordinates into a map of points:

In [ ]:
airports=gpd.GeoDataFrame(data=infoairports.copy(),
                 geometry=gpd.points_from_xy(infoairports.longitude_deg,
                                             infoairports.latitude_deg), 
                 crs=countryGit.crs.to_epsg())# the coordinates were in degrees - unprojected

In [ ]:
# does it look better?

# let's plot

base = countryGit.plot(color='white', edgecolor='black')
airports.plot(ax=base)

In [ ]:
#remember:
type(airports), type(infoairports)

Then this works:

In [ ]:
airports.to_crs(5641).plot()

In [ ]:
# this does not:
infoairports.to_crs(5641).plot()

Let's keep the projected version:

In [ ]:
airports_5641=airports.to_crs(5641)

Remember you have type of airports:

In [ ]:
airports_5641['type'].value_counts() # this will not work: airports.type.value_counts()

We may use that in the future. For now, just rename the **type** column to a different one.

In [ ]:
airports_5641.rename(columns={'type':'kind'},inplace=True)

Now, let's add this map of points:

In [ ]:
airports_5641.to_file(os.path.join("maps","brazilMaps_5641.gpkg"),layer='airports', driver="GPKG")


### Exercise 4
<div class="alert-success">

1. Look for some points in a CSV file about your country.
    
2. Turn those points into a spatial object.
    
3. Update the geopackage with the Brazil layers with the layer of points.
    
</div>

<a class="anchor" id="4"></a>

## Checking Validity of Geometry

Geometries are created in a way that some issues may appear, especially in (multi) polygons.
Let's check if our recent maps on states and municipalities are valid:

In [ ]:
# non valid
brazil_states_5641[~brazil_states_5641.is_valid]

In [ ]:
# see the invalid:
brazil_states_5641[~brazil_states_5641.is_valid].plot()

It is difficult to see what is wrong. Let's get some information:

In [ ]:
# what is wrong?

from shapely.validation import explain_validity, make_valid

explain_validity(brazil_states_5641[~brazil_states_5641.is_valid].geometry)

Let's solve the issue:

In [ ]:
BrSt5641_valid=brazil_states_5641[~brazil_states_5641.is_valid].copy()

# solving the issue:
BrSt5641_valid['geometry'] = [make_valid(row)  if not row.is_valid else row for row in BrSt5641_valid['geometry']]


In [ ]:
# any invalid?
BrSt5641_valid[~BrSt5641_valid.is_valid]

What about the municipalities layer with more polygons?

In [ ]:
brazil_municipalities_5641[~brazil_municipalities_5641.is_valid]

In [ ]:
explain_validity(brazil_municipalities_5641[~brazil_municipalities_5641.is_valid].geometry)

In [ ]:
Validity=pd.Series([x.split('[')[0] for x in explain_validity(brazil_municipalities_5641.geometry)])
Validity.value_counts()

In [ ]:
# solving the issue:

BrMun5641_valid=brazil_municipalities_5641.copy()

BrMun5641_valid['geometry'] = [make_valid(row)  if not row.is_valid else row for row in BrMun5641_valid['geometry'] ]
#any invalid?
BrMun5641_valid[~BrMun5641_valid.is_valid]

The _solution_ we got may help for some advanced techniques, but may also give us some extra trouble. Notice that once geopandas solved the problem, you  have created **collections**:

In [ ]:
[x for x in BrMun5641_valid["geometry"]]

Let's not save these last changes.

### Exercise 5 

<div class="alert-success">

1. Check if all your polygons are valid in every map you have (map of polygons only).
    
2. If you detect some invalid geometries, detect what the problem is.
    
3. Correct all cases. Do not save the results.
    
</div>

### Exercise 6 

<div class="alert-success">

Open the geopackage file in R and use ggplot2 to see the maps.
    
</div>